In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchmetrics
from torchmetrics.functional import accuracy, f1_score

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

/usr/local/lib/python3.6/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


## Model Setup 

In [2]:
pl.seed_everything(42, workers=True)

class DrowsyDataset(Dataset):
    
    def __init__(self, sequences):
        self.sequences = sequences
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        sequence, label = self.sequences[idx]
        return dict(
            sequence=torch.Tensor(sequence.to_numpy()),
            label=torch.tensor(label).long()
        )
    
class DrowsyDataModule(pl.LightningDataModule):
    
    def __init__(self, train_sequences, val_sequences, test_sequences, batch_size):
        super().__init__()
        self.train_sequences = train_sequences
        self.val_sequences = val_sequences
        self.test_sequences = test_sequences
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.train_dataset = DrowsyDataset(self.train_sequences)
        self.val_dataset = DrowsyDataset(self.val_sequences)
        self.test_dataset = DrowsyDataset(self.test_sequences)
        
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=os.cpu_count()
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=os.cpu_count()
        )
    
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=os.cpu_count()
        ) 

class DrowsyModel(nn.Module):
    
    def __init__(self, n_features, n_classes, n_hidden=256, n_layers=3):
        super().__init__()
        
        self.n_hidden = n_hidden
        
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=n_hidden,
            num_layers=n_layers,
            batch_first=True,
            dropout=0.75
        )
        
        self.classifier = nn.Linear(n_hidden, n_classes)
        
    def forward(self, x):
        self.lstm.flatten_parameters()
        _, (hidden, _) = self.lstm(x)
        
        out = hidden[-1]
        return self.classifier(out)
    
class DrowsyPredictor(pl.LightningModule):
    
    def __init__(self, n_features: int, n_classes: int):
        super().__init__()
        self.model = DrowsyModel(n_features, n_classes)
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, x, label=None):
        output = self.model(x)
        loss = 0
        if label is not None:
            loss = self.criterion(output, label)
        return loss, output
        
    def training_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = accuracy(predictions, labels)
        my_f1_score = f1_score(predictions, labels)
        
        self.log("train_loss", loss, prog_bar=True, logger=True)
        self.log("train_accuracy", step_accuracy, prog_bar=True, logger=True)
        self.log("train_f1_score", my_f1_score, prog_bar=True, logger=True)
        
        return {"loss": loss, "accuracy": step_accuracy, "f1_score": my_f1_score}
    
    def validation_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = accuracy(predictions, labels)
        my_f1_score = f1_score(predictions, labels)
        
        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.log("val_accuracy", step_accuracy, prog_bar=True, logger=True)
        self.log("val_f1_score", my_f1_score, prog_bar=True, logger=True)
        
        return {"loss": loss, "accuracy": step_accuracy, "f1_score": my_f1_score}
    
    def test_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = accuracy(predictions, labels)
        my_f1_score = f1_score(predictions, labels)
        
        self.log("test_train_loss", loss, prog_bar=True, logger=True)
        self.log("test_train_accuracy", step_accuracy, prog_bar=True, logger=True)
        self.log("test_f1_score", my_f1_score, prog_bar=True, logger=True)
        
        return {"loss": loss, "accuracy": step_accuracy, "f1_score": my_f1_score}
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.0001)

Global seed set to 42


## 參數設置

In [3]:
%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir ./lightning_logs --host 0.0.0.0 --port=8888

In [4]:
X_train = pd.read_csv('../data2.csv')

In [5]:
N_EPOCHS = 200
BATCH_SIZE = 50

# ORG_FEATURE_COLUMNS = ['TP8', 'FP2', 'FCZ', 'FT10', 'O2', 'O1', 'FT7', 'F4', 'TP7', 'C3', 'C4', 'F3', 'FT8', 'T6', 'HEOR', 'T5', 'VEOL', 'F7', 'FZ', 'VEOU', 'A1', 'P3', 'PZ', 'CP3', 'P4', 'CPZ', 'A2', 'HEOL', 'CP4', 'FT9', 'F8', 'OZ', 'CZ', 'FC4', 'FC3', 'FP1', 'PO1', 'T3', 'T4', 'PO2']
g = X_train.groupby("group")

ORG_FEATURE_COLUMNS = ['FP1','FP2','FZ']


## Main

In [ ]:
n = 0
FEATURE_COLUMNS = ORG_FEATURE_COLUMNS[n:]

while(n < 1):
    print("------------------ Round: " + str(n) + " ------------------")
    print(FEATURE_COLUMNS)
    print("Len:", len(FEATURE_COLUMNS))
    
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(X_train.state)

    label_encoder.classes_

    X_train['label'] = encoded_labels

    # Prepare data ###########################################
    sequences = [] 

    for name, group in g:
        sequence_features = group[FEATURE_COLUMNS]
        label = group.label.iloc[0]

        # print((sequence_features, label))
        sequences.append((sequence_features, label))

    # Setting up train, test, val gruop #######################
    train_sequences, test_sequences = train_test_split(sequences, test_size=0.2)
    val_sequences, test_sequences = train_test_split(test_sequences, test_size=0.5)


    # Setting up data module ##################################
    data_module = DrowsyDataModule(
        train_sequences, val_sequences, test_sequences, BATCH_SIZE
    )

    model = DrowsyPredictor(
        n_features=len(FEATURE_COLUMNS), 
        n_classes=len(label_encoder.classes_)
    )

    # Check points and logger #################################
    checkpoint_callback = ModelCheckpoint(
        dirpath="checkpoints",
        filename="best-checkpoint-corr-forhead-" + str(n),
        save_top_k=2,
        verbose=True,
        monitor="val_loss",
        mode="min"
    )

    logger = TensorBoardLogger("lightning_logs", name="Drowsy_" + str(n))

    trainer = pl.Trainer(
        logger=logger,
        callbacks=[checkpoint_callback],
        max_epochs=N_EPOCHS,
        gpus=[1],
        # gpus=2, 
        # auto_select_gpus=True,
        auto_lr_find=True, 
        # check_val_every_n_epoch=10
        # refresh_rate=20,
    )
    
    # Training start
    if (n == 41):
        trainer.fit(model, data_module, ckpt_path="/Workspace/code/checkpoints/best-checkpoint-corr-0-v1.ckpt")
    else:
        trainer.fit(model, data_module)
    trainer.test(model, data_module)
    
    FEATURE_COLUMNS.pop(0)
    n = n + 1

------------------ Round: 0 ------------------
['FP1', 'FP2', 'FZ']
Len: 3


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | DrowsyModel      | 1.3 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.282     Total estimated model params size (MB)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /Workspace/code/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Global seed set to 42


Epoch 0:  89%|████████▊ | 226/255 [02:16<00:17,  1.66it/s, loss=0.678, v_num=8, train_loss=0.683, train_accuracy=0.611, train_f1_score=0.611]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 255/255 [02:23<00:00,  1.78it/s, loss=0.678, v_num=8, train_loss=0.683, train_accuracy=0.611, train_f1_score=0.611, val_loss=0.673, val_accuracy=0.566, val_f1_score=0.566]

Epoch 0, global step 225: val_loss reached 0.67300 (best 0.67300), saving model to "/Workspace/code/checkpoints/best-checkpoint-corr-forhead-0.ckpt" as top 2


...

Epoch 35:  89%|████████▊ | 226/255 [02:22<00:18,  1.59it/s, loss=0.633, v_num=8, train_loss=0.629, train_accuracy=0.667, train_f1_score=0.667, val_loss=0.625, val_accuracy=0.624, val_f1_score=0.624]
Validating: 0it [00:00, ?it/s]
Epoch 35: 100%|██████████| 255/255 [02:29<00:00,  1.71it/s, loss=0.633, v_num=8, train_loss=0.629, train_accuracy=0.667, train_f1_score=0.667, val_loss=0.626, val_accuracy=0.609, val_f1_score=0.609]

Epoch 35, global step 8135: val_loss was not in top 2


Epoch 36:  89%|████████▊ | 226/255 [02:22<00:18,  1.59it/s, loss=0.636, v_num=8, train_loss=0.844, train_accuracy=0.556, train_f1_score=0.556, val_loss=0.626, val_accuracy=0.609, val_f1_score=0.609]
Validating: 0it [00:00, ?it/s]
Epoch 36: 100%|██████████| 255/255 [02:29<00:00,  1.71it/s, loss=0.636, v_num=8, train_loss=0.844, train_accuracy=0.556, train_f1_score=0.556, val_loss=0.627, val_accuracy=0.616, val_f1_score=0.616]

Epoch 36, global step 8361: val_loss was not in top 2


Epoch 37:  53%|█████▎    | 134/255 [01:21<01:13,  1.64it/s, loss=0.674, v_num=8, train_loss=0.788, train_accuracy=0.500, train_f1_score=0.500, val_loss=0.627, val_accuracy=0.616, val_f1_score=0.616]

## Load Checkpoint 

In [ ]:
# best-checkpoint-corr-3-v1.ckptn
# trainer.fit(model, data_module, ckpt_path="/Workspace/code/checkpoints/best-checkpoint-corr-3-v1.ckpt")